<a href="https://colab.research.google.com/github/Jack3690/pydol/blob/main/notebooks/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/Jack3690/pydol/ -q
!pip install astroquery -q

  Preparing metadata (setup.py) ... done


# **DOLPHOT Installation**

In [ ]:
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.tar.gz
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.NIRCAM.tar.gz
!wget http://americano.dolphinsim.com/dolphot/nircam_F140M.tar.gz
!wget http://americano.dolphinsim.com/dolphot/nircam_F212N.tar.gz

In [3]:
!tar -xf dolphot2.0.tar.gz
!tar -xf dolphot2.0.NIRCAM.tar.gz
!tar -xf nircam_F140M.tar.gz
!tar -xf nircam_F212N.tar.gz

In [4]:
with open('/content/dolphot2.0/Makefile') as f:
  dat = f.readlines()
dat[28]= dat[28][1:]

with open('/content/dolphot2.0/Makefile','w') as f:
  f.writelines(dat)

In [5]:
cd /content/dolphot2.0/

/content/dolphot2.0


In [6]:
import os

In [7]:
os.system('make all')

0

In [8]:
cd /content/

/content


In [11]:
os.environ['PATH'] += ':/content/dolphot2.0/bin'
os.environ['PATH'] += ':/content/dolphot2.0/nircam'

# **Imports**

In [6]:
from glob import glob
import astropy.io.fits as fits

import os
import astroquery
from astroquery.mast import Observations
import pandas as pd

from astropy.table import Table
from astropy.wcs import WCS
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import json
from astropy.coordinates import angular_separation
import astropy.units as u
from pydol.pipeline.jwst import jpipe
import multiprocessing as mp

# **Data**

**Observation ID format**

jw\<ppppp>\<ooo><vvv>\_\<gg>\<s>\<aa>\_\<eeeee>(-<"seg"NNN>)\_\<detector>\_\<prodType>.fits

where

ppppp: program ID number

ooo: observation number

vvv: visit number

gg: visit group

s: parallel sequence ID (1=prime, 2-5=parallel)

aa: activity number (base 36)

eeeee: exposure number

segNNN: the text “seg” followed by a three-digit segment number (optional)

detector: detector name (e.g. ‘nrca1’, ‘nrcblong’, ‘mirimage’)

prodType: product type identifier (e.g. ‘uncal’, ‘rate’, ‘cal’)

In [ ]:
#Example data for M82

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/data/stage0/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

ext = 'uncal'
# JWST images to be analyzed
image_files = []

# ngc 628: f"jw0178390400{k}_02101_0000{i}
for i in range(1,10):
    for j in range(1,5):
            for l in ['b']:
                image_files.append( f"jw01701052001_02102_0000{i}_nrc{l}{j}_uncal.fits")

for image_file in image_files:
    # Download file (if not already downloaded)
    mast_path  = os.path.join(mast_dir, image_file)
    local_path = os.path.join(data_dir, image_file)
    Observations.download_file(mast_path, local_path=local_path)

In [41]:
#Example data for M82

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/photometry/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

# JWST images to be analyzed
image_files = [ f"jw01701-o052_t007_nircam_clear-f140m-sub640_i2d.fits"]

for image_file in image_files:
    # Download file (if not already downloaded)
    mast_path  = os.path.join(mast_dir, image_file)
    local_path = os.path.join(data_dir, image_file)
    Observations.download_file(mast_path, local_path=local_path)

# **Pipeline**

In [ ]:
input_files = glob(f'/content/data/stage0/*')
out_dir = '/content/'
jwst_data = jpipe(input_files, out_dir)
jwst_data()

In [8]:
cal_files = glob(f'/content/data/stage2/*cal.fits')

In [ ]:
cal_files

# **DOLPHOT Photometry**

In [72]:
!rm -r /usr/local/lib/python3.10/dist-packages/pydol

rm: cannot remove '/usr/local/lib/python3.10/dist-packages/pydol': No such file or directory


In [71]:
!pip uninstall pydol

In [3]:
from pydol.photometry import nircam

In [9]:
phot = nircam.nircam_phot(cal_files, 'f140m','/content/photometry/',
                          '/content/photometry/jw01701-o052_t007_nircam_clear-f140m-sub640_i2d')

FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/pydol/photometry/params/nircam_dolphot.param'

In [45]:
import subprocess

  and should_run_async(code)

2024-06-28 00:00:40,635 - stpipe - WARNING - /usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



In [56]:
out= subprocess.run(['nircammask', '/content/photometry/jw01701-o052_t007_nircam_clear-f140m-sub640_i2d.fits'
],stdout= subprocess.PIPE)

In [57]:
out.stdout

b'Reading FITS file /content/photometry/jw01701-o052_t007_nircam_clear-f140m-sub640_i2d.fits\nReading IMAGE extension: 1644x1634\n**Neither DQ nor WHT extension found\n'

In [59]:
subprocess.check_output(out)

  and should_run_async(code)

2024-06-28 00:07:15,239 - stpipe - WARNING - /usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



TypeError: 'CompletedProcess' object is not iterable